In [1]:
import sys
from pathlib import Path

In [2]:
# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import pandas as pd
from config import output_path_prefix

In [19]:
df_eval = pd.read_csv(f"{output_path_prefix}_eval.csv")

In [20]:
len(df_eval)

100

In [21]:
df_eval.head(1)

,Unnamed: 0.1,Unnamed: 0,query,answer,page_number,outputs.answer,outputs.page_number,retrieved_contexts
0,0,0,"전 세계에서 초중고 CS 교육을 제공하거나 제공할 계획인 국가의 비율은 얼마이며, ...","전 세계 국가 중 약 3분의 2가 초중고 CS 교육을 제공하거나 제공할 계획이며, ...",21,요약(작성자)와 원본(작성자)\n- 요약 보고서 작성자: SPRi (SPRi 이슈리...,"[21, 22]",### Context #1<document><page_content>SPRi 이슈리...


In [25]:
queries = df_eval['query'].tolist()
responses = df_eval['outputs.answer'].tolist()
references = df_eval['answer'].tolist()
relevant_docs = df_eval['retrieved_contexts'].tolist()

In [29]:
dataset = []
for query, response, reference, relevant_doc in zip(queries, responses, references, relevant_docs):
    dataset.append({
        "user_input": query,
        "response": response,
        "reference": reference,
        "retrieved_contexts": [c.strip() for c in relevant_doc.split("### Context") if c.strip()]
    })



In [31]:
dataset[0]['retrieved_contexts']

['#1<document><page_content>SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\n7. 교육\n=====\n\n⊙ 세계적으로 AI 및 전산학(Computer Science, CS)8) 교육이 빠르게 확산되고  \n있으며, AI 전문가 배출도 가속화\n\n  \n\n⇨ 전 세계 국가 중 3분의 2가 초중고 CS 교육을 제공하거나 제공할 계획  \n⇨ 미국의 경우, 고등학교 CS 과정에 대한 접근성과 등록률은 이전 학년도보다 약간 증가  \n했지만 여전히 격차가 존재\n\n  \n\n※ 연구진은 교육용 AI, AI 역량, AI 교육 등 국가·인종별 전산학 및 AI 교육과정 이용 현황 조사하고,  \nK-12 과정의 교육 프레임워크 도입과 고등교육과정에서의 인재 양성 지수를 분석\n\n□ (AI 교육 도입 확산) ECEP9)에서 인용된 미국 고등학교의 CS 과정을  \n포함하는 비율은 2017년에는 35%에서 `23년 60%로 증가하여 기초 CS  \n과정의 중요성을 강조하고 있음\n\n![](/images/SPRI_2025_cropped_figure_248.png)\n\n※ 자료: Standford HAI, AI Index Report 2025\n\n  \n\n[그림 7-1] 국가별 CS 교육 제공 여부 맵\n\n  \n\n◦ (CS 교육 도입) `24년에는 전 세계 국가 중 약 2/3에 해당하는 국가가 CS  \n과정을 도입하거나 도입할 예정\n\n  \n\n\xad 대한민국은 가나, 네덜란드 등과 함께 AI 교육 내용을 공식 커리큘럼에  \n명시한 소수 국가 중 하나로 인식\n\n8) AI는 전통적으로 전산학(CS)의 하위 분야로 연구되어 왔기 때문에, 본 장에서는 AI 관련 자료가 없는 경우 CS 교육  \n데이터를 참고하여 서술  \n9) ECEP(Expanding Computing Education Pathways)는 2012년도에 미국 조지아 공과대학의 지원 아래 설립된 단체로,  \

In [32]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [37]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [44]:
from ragas import evaluate
from ragas.llms import llm_factory
from openai import OpenAI

llm = llm_factory('gpt-5-mini', client=OpenAI())
from ragas.metrics.collections import ContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,metrics=[ContextRecall(llm), Faithfulness(llm), FactualCorrectness(llm)], llm=llm)
result

TypeError: All metrics must be initialised metric objects, e.g: metrics=[BleuScore(), AspectCritic()]